# <font color='magenta'>API Wrapper</font>

In [6]:
"""
Read in genes of interest
"""

filename = 'wheat_test_2'

with open(f'{filename}.txt') as f:
    # remove trailing '\n'
    genes = f.read().splitlines()

In [7]:
import urllib.request
from bs4 import BeautifulSoup

class Denim(object):
    """
    Denim Class
    ~~~~~~~~~~~~
    Extracts mirna interactions from a target gene 
    and returns a parsed list of dicts
    Private methods build request url and parse returned data 
    
    get_genes  : 
        Input:
            geneID : str : tair10 gene identifier eg 'AT2G33810'
        returns:
            [list of dicts]
            data return in following format:
        
            [{'miRNA' : <miRNA string>, 'target_gene' : <target gene string>},
            { ... },
            ...
            ]
    """
    
    def __init__(self):
        self.url = 'http://www.pmiren.com/browsehref4search?stype=61&'
        
    def _build_url(self, geneID):
        return f"{self.url}kw={geneID}"
    
    def _make_soup(self, html : str):
        soup = BeautifulSoup(html, 'html5lib')
        return soup
    
    def _parse(self, soup):
        """
        Explain format of html
        """
        rnas = []
        entries = [entry for entry in soup.find_all('tr')]
        # skip first entry - headers
        for entry in entries[1:]:    
            elements = [element for element in entry.find_all('td')]
            
            rnas.append({'miRNA': elements[0].string,
                        'target_gene' : elements[1].string}) 
        
        return rnas
    
    def get_genes(self, geneID : str):
        """
        """
        rqst_url = self._build_url(geneID)
        response = urllib.request.urlopen(rqst_url)
        soup = self._make_soup(response.read())
        
        return self._parse(soup)

In [ ]:
wrapper = Denim()

tofile = []

for gene in genes[1:]:
    tofile.extend(wrapper.get_genes(gene))
        

In [ ]:
tofile

In [5]:
import csv
keys = tofile[1].keys()
with open ('example.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(tofile)